In [1]:
'''Import libraries'''
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification, AdamW, Trainer, TrainingArguments
from tqdm import tqdm
from torch.nn import functional as F
import torch.nn as nn

wandb.login()

c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alberto-rodero557. Use `wandb login --relogin` to force relogin


True

In [2]:
'''Variables and parameters'''

SAMPLES_TO_TRAIN=10000
DIMENSIONS=200

N_LABELS=2
MAX_LEN = 256
EPOCHS=100
PATIENCE=10
LEARNING_RATE=.00005
WEIGHT_DECAY=.01
BATCH_SIZE=16
METRIC_FOR_BEST_MODEL='eval_loss'
if METRIC_FOR_BEST_MODEL=='eval_loss':
    GREATER_IS_BETTER = False
else:
    GREATER_IS_BETTER = True

In [3]:
'''Preparing dataset'''

df = pd.read_json(os.getcwd()+'/datasets/subtaskA_train_monolingual.jsonl', lines=True)
df = df[['text', 'label']]

df=df.sample(round(SAMPLES_TO_TRAIN))
# test_train_df=df.sample(round(SAMPLES_TO_TRAIN*.2))

# df = pd.read_json(os.getcwd()+'/datasets/subtaskA_dev_monolingual.jsonl', lines=True)
# df = df[['text', 'label']]

# val_df= df.sample(round(SAMPLES_TO_TRAIN*.2))
# test_dev_df= df.sample(round(SAMPLES_TO_TRAIN*.2))

# we balance the training set
print(f'Dataset size before balancing: {df.shape}')
counts = df['label'].value_counts()
sampler = RandomUnderSampler(random_state=42)
x_text, y = sampler.fit_resample(df[['text']], df['label'])

print(f'Dataset size after balancing: {x_text.shape}')
print(f'Entried dropped: {df.shape[0]-x_text.shape[0]}')

# Create a new balanced DataFrame
df = pd.DataFrame({'text': x_text['text'], 'label': y})

# Print the balanced DataFrame
print("\nBalanced DataFrame:")
print(df['label'].value_counts())

Dataset size before balancing: (10000, 2)
Dataset size after balancing: (9264, 1)
Entried dropped: 736

Balanced DataFrame:
label
0    4632
1    4632
Name: count, dtype: int64


In [4]:
'''loading glove'''
embeddings_index={}
with open('../0 playground and indoor/OtherData/glove.6B.200d.txt','r',encoding='utf-8') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embeddings_index[word]=vectors
f.close()
print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


In [5]:
'''glove building'''

from nltk.tokenize import word_tokenize
from tqdm import tqdm 

def sent2vec(s):
    """ Function Creates a normalized vector for the whole sentence"""
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(200)
    return v / np.sqrt((v ** 2).sum())

print('Training df:')
df_x = np.array([sent2vec(x) for x in tqdm(df['text'])])
print(df_x.shape)
train_y=df['label']


Training df:


100%|██████████| 9264/9264 [00:12<00:00, 726.20it/s] 

(9264, 200)


In [6]:
'''Preparing for training'''

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Initialize the StandardScaler
scaler = StandardScaler()
# Fit the scaler to the training data and transform the data
train_x = scaler.fit_transform(df_x)

import pickle

# Save the trained scaler
with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)


In [7]:
'''metrics'''

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'auc': auc,
        'precision': precision,
        'recall': recall,
    }

In [8]:
class Data(Dataset):
    def __init__(self, X_train, y_train):
        self.X = torch.from_numpy(X_train.astype(np.float32))
        self.y = torch.from_numpy(y_train).type(torch.LongTensor)
        self.len = self.X.shape[0]

    def __getitem__(self, index):
        return {'input_ids': self.X[index], 'labels': self.y[index]}

    def __len__(self):
        return self.len

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y.values, test_size=0.2, random_state=42)
traindata = Data(X_train, y_train)
testdata = Data(X_test, y_test)

In [28]:
# number of features (len of X cols)
input_dim = train_x.shape[-1]

# number of classes (unique of y)
output_dim = 2

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.linear1 = nn.Linear(input_dim, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.dropout1 = nn.Dropout(0.5)
        
        self.linear2 = nn.Linear(512, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.dropout2 = nn.Dropout(0.5)
        
        self.linear3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout3 = nn.Dropout(0.5)
        
        self.linear4 = nn.Linear(256, output_dim)
        
        self.loss = nn.CrossEntropyLoss()

    def forward(self, input_ids, labels=None):
        x1 = F.leaky_relu(self.linear1(input_ids))
        x1 = self.bn1(x1)
        x1 = self.dropout1(x1)
        
        x2 = F.leaky_relu(self.linear2(x1))
        x2 = self.bn2(x2)
        x2 = self.dropout2(x2)
        
        # Adding the first skip connection
        x2 += x1
        
        x3 = F.leaky_relu(self.linear3(x2))
        x3 = self.bn3(x3)
        x3 = self.dropout3(x3)
        
        x4 = self.linear4(x3)
        
        outputs = (x4,)
        if labels is not None:
            loss = self.loss(x4, labels)
            outputs = (loss,) + outputs
            
        return (outputs if len(outputs) > 1 else outputs[0])

# Create the model
model = Network()

In [29]:
from transformers import EarlyStoppingCallback

model = Network()

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=500,
    weight_decay=WEIGHT_DECAY,
    metric_for_best_model=METRIC_FOR_BEST_MODEL,
    greater_is_better=GREATER_IS_BETTER,
    logging_dir='./logs',
    logging_steps=15000,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=False,
    logging_first_step=False,
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="wandb"
)

# Create trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=traindata,             # training dataset
    eval_dataset=testdata, 
    compute_metrics=compute_metrics,# training dataset
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)]
)

# Train the model
print(trainer.evaluate())

trainer.train()

print(trainer.evaluate())

100%|██████████| 116/116 [00:00<00:00, 998.91it/s] 


{'eval_loss': 0.696049690246582, 'eval_accuracy': 0.49487317862924984, 'eval_f1': 0.11698113207547171, 'eval_auc': 0.49189500908709627, 'eval_precision': 0.44285714285714284, 'eval_recall': 0.06739130434782609, 'eval_runtime': 0.1171, 'eval_samples_per_second': 15820.313, 'eval_steps_per_second': 990.37}


  1%|          | 528/46400 [00:01<02:24, 317.11it/s]

{'eval_loss': 0.5957270860671997, 'eval_accuracy': 0.6729627630868862, 'eval_f1': 0.6464410735122521, 'eval_auc': 0.6724695931776876, 'eval_precision': 0.6977329974811083, 'eval_recall': 0.6021739130434782, 'eval_runtime': 0.1106, 'eval_samples_per_second': 16748.726, 'eval_steps_per_second': 1048.49, 'epoch': 1.0}


  2%|▏         | 976/46400 [00:02<02:24, 314.44it/s]

...

{'eval_loss': 0.4016591012477875, 'eval_accuracy': 0.8192120885051268, 'eval_f1': 0.8135781858653311, 'eval_auc': 0.819040379328021, 'eval_precision': 0.8335233751425314, 'eval_recall': 0.7945652173913044, 'eval_runtime': 0.1133, 'eval_samples_per_second': 16355.039, 'eval_steps_per_second': 1023.845, 'epoch': 45.0}


 46%|████▌     | 21344/46400 [01:00<01:11, 350.88it/s]


{'eval_loss': 0.4066920876502991, 'eval_accuracy': 0.8202914193200216, 'eval_f1': 0.8214477211796246, 'eval_auc': 0.8203772309986486, 'eval_precision': 0.8105820105820106, 'eval_recall': 0.8326086956521739, 'eval_runtime': 0.1147, 'eval_samples_per_second': 16158.334, 'eval_steps_per_second': 1011.531, 'epoch': 46.0}
{'train_runtime': 60.83, 'train_samples_per_second': 12183.142, 'train_steps_per_second': 762.782, 'train_loss': 0.45767735791528064, 'epoch': 46.0}


100%|██████████| 116/116 [00:00<00:00, 943.12it/s]

{'eval_loss': 0.400961309671402, 'eval_accuracy': 0.8154344306529951, 'eval_f1': 0.8069977426636569, 'eval_auc': 0.8151678782795098, 'eval_precision': 0.8392018779342723, 'eval_recall': 0.7771739130434783, 'eval_runtime': 0.1245, 'eval_samples_per_second': 14884.328, 'eval_steps_per_second': 931.777, 'epoch': 46.0}


In [33]:
# 200 100 100 2
# bn and dp 0.3
# 'eval_loss': 0.41982632875442505, 'eval_accuracy': 0.8111171073934161, 'eval_f1': 0.8046875 epoch 79

# 100 to 256
# eval_loss': 0.40763354301452637, 'eval_accuracy': 0.813815434430653, 'eval_f1': 0.807585052983826

# leaky relu
# 'eval_loss': 0.41205745935440063, 'eval_accuracy': 0.8235294117647058, 'eval_f1': 0.8229561451001625

# 512 and dp 0.5
# 

100%|██████████| 250/250 [00:00<00:00, 1069.73it/s]


{'eval_loss': 0.6834174990653992,
 'eval_accuracy': 0.522,
 'eval_f1': 0.0020876826722338207,
 'eval_auc': 0.5005224660397074,
 'eval_precision': 1.0,
 'eval_recall': 0.0010449320794148381,
 'eval_runtime': 0.2357,
 'eval_samples_per_second': 8485.118,
 'eval_steps_per_second': 1060.64,
 'epoch': 9.0}